## Aim: To create final training and test dataframes with new & important features
# Download data

In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
!rm train_etl.csv test_etl.csv pairs_dict.pkl connectivity_model_nn_03.h5
!ls

rm: cannot remove ‘train_etl.csv’: No such file or directory
rm: cannot remove ‘test_etl.csv’: No such file or directory
rm: cannot remove ‘pairs_dict.pkl’: No such file or directory
rm: cannot remove ‘connectivity_model_nn_03.h5’: No such file or directory


In [3]:
download_file_cos(credentials_2, 'train_etl.csv', 'train_etl.csv')
download_file_cos(credentials_2, 'test_etl.csv', 'test_etl.csv')
download_file_cos(credentials_2, 'pairs_dict.pkl', 'pairs_dict.pkl')
download_file_cos(credentials_2, 'connectivity_model_nn_03.h5', 'connectivity_model_nn_03.h5')
!ls

File Downloaded
File Downloaded
File Downloaded
File Downloaded
connectivity_model_nn_03.h5  pairs_dict.pkl  test_etl.csv  train_etl.csv


In [4]:
#QM9
!wget https://www.kaggleusercontent.com/kf/15612312/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..XpvpY1AwxFHBalEO6EW75w.4xvQ52UpYKPAQp5lmOt_cSCeO_zDbr49lhqEr-LSneu9R5C_de3BtMl2KyXRR6uTUcwCzSfRb4vrmg5kFhzFjfg91HmxgfBTSuwrK3MBA62w4S8P51t083QEE4RXibSirX11HYn7eAph-MFMSHJ_yA.r8PDG6MhWtotvYsMKAnlnA/data.covs.pickle

--2019-08-14 09:58:00--  https://www.kaggleusercontent.com/kf/15612312/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..XpvpY1AwxFHBalEO6EW75w.4xvQ52UpYKPAQp5lmOt_cSCeO_zDbr49lhqEr-LSneu9R5C_de3BtMl2KyXRR6uTUcwCzSfRb4vrmg5kFhzFjfg91HmxgfBTSuwrK3MBA62w4S8P51t083QEE4RXibSirX11HYn7eAph-MFMSHJ_yA.r8PDG6MhWtotvYsMKAnlnA/data.covs.pickle
Resolving www.kaggleusercontent.com (www.kaggleusercontent.com)... 35.190.26.106
Connecting to www.kaggleusercontent.com (www.kaggleusercontent.com)|35.190.26.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1678972335 (1.6G) [application/octet-stream]
Saving to: ‘data.covs.pickle’

100%[====================================>] 1,678,972,335  119MB/s   in 14s    

2019-08-14 09:58:15 (113 MB/s) - ‘data.covs.pickle’ saved [1678972335/1678972335]



In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
from tensorflow import keras

%matplotlib inline

In [6]:
tf.__version__

'1.13.1'

In [7]:
train=pd.read_csv('train_etl.csv')
test=pd.read_csv('test_etl.csv')
print('train:{}'.format(train.shape))
print('test: {}'.format(test.shape))

train:(4658147, 16)
test: (2505542, 15)


In [8]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074


In [9]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,2JHC
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,1JHC
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,NaN,3JHH
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,1JHC
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,2JHC


In [10]:
train.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,4.65815e+06,NaN,NaN,NaN,2.32907e+06,1.34469e+06,0,1.16454e+06,2.32907e+06,3.49361e+06,4.65815e+06
molecule_name,4658147,85003,dsgdb9nsd_042139,135,NaN,NaN,NaN,NaN,NaN,NaN,NaN
atom_index_0,4.65815e+06,NaN,NaN,NaN,13.3569,3.26771,0,11,13,16,28
atom_index_1,4.65815e+06,NaN,NaN,NaN,5.88397,4.99394,0,2,5,8,28
atom_0,4658147,1,H,4658147,NaN,NaN,NaN,NaN,NaN,NaN,NaN
x_0,4.65815e+06,NaN,NaN,NaN,0.105878,1.75987,-9.23489,-0.953092,0.134121,1.22775,9.38224
y_0,4.65815e+06,NaN,NaN,NaN,-0.220369,2.12179,-9.49416,-1.83697,-0.297994,1.75342,9.71447
z_0,4.65815e+06,NaN,NaN,NaN,0.0471289,1.56451,-9.13476,-0.964213,0.00497289,1.0351,7.63758
atom_1,4658147,3,C,3360469,NaN,NaN,NaN,NaN,NaN,NaN,NaN
x_1,4.65815e+06,NaN,NaN,NaN,0.0975718,1.47942,-9.23489,-0.79276,0.0466042,0.968224,8.42089


In [11]:
test.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,2.50554e+06,NaN,NaN,NaN,5.91092e+06,723288,4.65815e+06,5.28453e+06,5.91092e+06,6.5373e+06,7.16369e+06
molecule_name,2505542,45772,dsgdb9nsd_060445,131,NaN,NaN,NaN,NaN,NaN,NaN,NaN
atom_index_0,2.50554e+06,NaN,NaN,NaN,13.3545,3.27221,1,11,13,16,28
atom_index_1,2.50554e+06,NaN,NaN,NaN,5.87795,4.99231,0,2,5,8,28
atom_0,2505542,1,H,2505542,NaN,NaN,NaN,NaN,NaN,NaN,NaN
x_0,2.50554e+06,NaN,NaN,NaN,0.111877,1.75132,-8.35577,-0.946371,0.139294,1.22997,9.15701
y_0,2.50554e+06,NaN,NaN,NaN,-0.223317,2.1128,-9.93394,-1.83265,-0.300268,1.74771,10.182
z_0,2.50554e+06,NaN,NaN,NaN,0.0532973,1.56707,-7.82342,-0.960673,0.00780067,1.04035,7.89473
atom_1,2505542,3,C,1805746,NaN,NaN,NaN,NaN,NaN,NaN,NaN
x_1,2.50554e+06,NaN,NaN,NaN,0.102163,1.4705,-8.35577,-0.775903,0.0472481,0.968822,8.18919


# Feature Engineering (train)

## Euclidian Distance

In [13]:
def get_distance(df):
    return np.sqrt(np.square(df['x_0']-df['x_1'])+np.square(df['y_0']-df['y_1'])+np.square(df['z_0']-df['z_1']))

In [14]:
train['distance']=train.apply(get_distance,axis=1)
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952


## one-hot  encode atom names
(Not neccessary)

In [14]:
#train=train.join(pd.get_dummies(train['atom_0'],prefix='atom_0'))
#train.head()

In [15]:
#train=train.join(pd.get_dummies(train['atom_1'],prefix='atom_1'))
#train.head()

## Label Encode type

In [20]:
from sklearn.preprocessing import LabelEncoder
le_type=LabelEncoder()
train['type_enc']=le_type.fit_transform(train['type'])

In [21]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953,0
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120,3
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147,3
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157,3
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952,0


In [18]:
train.corr().loc['scalar_coupling_constant']

id                         -0.006968
atom_index_0                0.018839
atom_index_1               -0.218989
x_0                        -0.006015
y_0                        -0.013119
z_0                         0.004807
x_1                         0.005132
y_1                         0.021189
z_1                        -0.007792
mulliken_charge_0          -0.025444
mulliken_charge_1          -0.360745
scalar_coupling_constant    1.000000
distance                   -0.668663
type_enc                   -0.641988
Name: scalar_coupling_constant, dtype: float64

id                         -0.006968
atom_index_0                0.018839
atom_index_1               -0.218989
x_0                        -0.006015
y_0                        -0.013119
z_0                         0.004807
x_1                         0.005132
y_1                         0.021189
z_1                        -0.007792
mulliken_charge_0          -0.025444
mulliken_charge_1          -0.360745
scalar_coupling_constant    1.000000
distance                   -0.668663
type_enc                   -0.641988
Name: scalar_coupling_constant, dtype: float64

## Pairs

In [19]:
train['pairs']=(train['atom_0']+'_'+train['atom_1']).apply(lambda x:x.split('_')).apply(sorted).apply(lambda x:'_'.join(x)).apply(str.lower)

In [20]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,pairs
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953,0,c_h
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120,3,h_h
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147,3,h_h
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157,3,h_h
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952,0,c_h


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,pairs
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953,0,c_h
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120,3,h_h
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147,3,h_h
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157,3,h_h
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952,0,c_h


## Label Encode Pairs using dictionary

In [21]:
!ls

connectivity_model_nn_03.h5  pairs_dict.pkl  test_etl.csv  train_etl.csv
connectivity_model_nn_03.h5  pairs_dict.pkl  test_etl.csv  train_etl.csv


In [22]:
with open('pairs_dict.pkl','rb') as f:
    pairs_dict=pickle.load(f)

In [23]:
pairs_dict

{'c_c': 0,
 'c_f': 1,
 'c_h': 2,
 'c_n': 3,
 'c_o': 4,
 'c_si': 5,
 'f_f': 6,
 'f_h': 7,
 'f_n': 8,
 'f_o': 9,
 'f_si': 10,
 'h_h': 11,
 'h_n': 12,
 'h_o': 13,
 'h_si': 14,
 'n_n': 15,
 'n_o': 16,
 'o_o': 17}

{'c_c': 0,
 'c_f': 1,
 'c_h': 2,
 'c_n': 3,
 'c_o': 4,
 'c_si': 5,
 'f_f': 6,
 'f_h': 7,
 'f_n': 8,
 'f_o': 9,
 'f_si': 10,
 'h_h': 11,
 'h_n': 12,
 'h_o': 13,
 'h_si': 14,
 'n_n': 15,
 'n_o': 16,
 'o_o': 17}

In [24]:
train['pairs'].unique()

array(['c_h', 'h_h', 'h_n'], dtype=object)

array(['c_h', 'h_h', 'h_n'], dtype=object)

In [25]:
train['pairs_enc']=train['pairs'].map(pairs_dict)

In [26]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,pairs,pairs_enc
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953,0,c_h,2
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120,3,h_h,11
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147,3,h_h,11
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157,3,h_h,11
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952,0,c_h,2


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,pairs,pairs_enc
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953,0,c_h,2
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120,3,h_h,11
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147,3,h_h,11
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157,3,h_h,11
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952,0,c_h,2


## Connectivity

In [27]:
model_nn=tf.keras.models.load_model('connectivity_model_nn_03.h5',custom_objects=None,compile=True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [28]:
x=train[['pairs_enc','distance']]

In [29]:
x.head()

,pairs_enc,distance
0,2,1.091953
1,11,1.783120
2,11,1.783147
3,11,1.783157
4,2,1.091952


,pairs_enc,distance
0,2,1.091953
1,11,1.783120
2,11,1.783147
3,11,1.783157
4,2,1.091952


In [30]:
x.shape

(4658147, 2)

(4658147, 2)

In [31]:
#model_nn.predict(x,verbose=1)

In [32]:
train['bond']=model_nn.predict(x,verbose=1,batch_size=10000)

4658147/4658147 [==============================] - 93s 20us/sample


In [33]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,pairs,pairs_enc,bond
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,...,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953,0,c_h,2,0.919760
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,...,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120,3,h_h,11,0.648310
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,...,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147,3,h_h,11,0.648205
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,...,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157,3,h_h,11,0.648170
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,...,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952,0,c_h,2,0.919760


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,pairs,pairs_enc,bond
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,...,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953,0,c_h,2,0.919760
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,...,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120,3,h_h,11,0.648310
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,...,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147,3,h_h,11,0.648205
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,...,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157,3,h_h,11,0.648170
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,...,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952,0,c_h,2,0.919760


In [34]:
train['bond'].isna().value_counts()

False    4658147
Name: bond, dtype: int64

False    4658147
Name: bond, dtype: int64

In [35]:
train['bond_int']=train['bond'].apply(lambda x:int(x>0.5))

In [36]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,pairs,pairs_enc,bond,bond_int
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,...,0.133921,-0.535689,1JHC,84.8076,1.091953,0,c_h,2,0.919760,1
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,...,0.133921,0.133922,2JHH,-11.2570,1.783120,3,h_h,11,0.648310,1
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,...,0.133921,0.133923,2JHH,-11.2548,1.783147,3,h_h,11,0.648205,1
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,...,0.133921,0.133923,2JHH,-11.2543,1.783157,3,h_h,11,0.648170,1
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,...,0.133922,-0.535689,1JHC,84.8074,1.091952,0,c_h,2,0.919760,1


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,pairs,pairs_enc,bond,bond_int
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,...,0.133921,-0.535689,1JHC,84.8076,1.091953,0,c_h,2,0.919760,1
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,...,0.133921,0.133922,2JHH,-11.2570,1.783120,3,h_h,11,0.648310,1
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,...,0.133921,0.133923,2JHH,-11.2548,1.783147,3,h_h,11,0.648205,1
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,...,0.133921,0.133923,2JHH,-11.2543,1.783157,3,h_h,11,0.648170,1
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,...,0.133922,-0.535689,1JHC,84.8074,1.091952,0,c_h,2,0.919760,1


## Total Charge

In [37]:
train['total_charge']=train['mulliken_charge_0']+train['mulliken_charge_1']

In [38]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,pairs,pairs_enc,bond,bond_int,total_charge
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,...,-0.535689,1JHC,84.8076,1.091953,0,c_h,2,0.919760,1,-0.401768
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,...,0.133922,2JHH,-11.2570,1.783120,3,h_h,11,0.648310,1,0.267843
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,...,0.133923,2JHH,-11.2548,1.783147,3,h_h,11,0.648205,1,0.267844
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,...,0.133923,2JHH,-11.2543,1.783157,3,h_h,11,0.648170,1,0.267844
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,...,-0.535689,1JHC,84.8074,1.091952,0,c_h,2,0.919760,1,-0.401767


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc,pairs,pairs_enc,bond,bond_int,total_charge
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,...,-0.535689,1JHC,84.8076,1.091953,0,c_h,2,0.919760,1,-0.401768
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,...,0.133922,2JHH,-11.2570,1.783120,3,h_h,11,0.648310,1,0.267843
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,...,0.133923,2JHH,-11.2548,1.783147,3,h_h,11,0.648205,1,0.267844
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,...,0.133923,2JHH,-11.2543,1.783157,3,h_h,11,0.648170,1,0.267844
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,...,-0.535689,1JHC,84.8074,1.091952,0,c_h,2,0.919760,1,-0.401767


## Dipole moment

sum(qi*ri)

In [39]:
train['dp']=(-1*train['mulliken_charge_0']*train['distance']/2)+(train['mulliken_charge_1']*train['distance']/2)

In [40]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,type,scalar_coupling_constant,distance,type_enc,pairs,pairs_enc,bond,bond_int,total_charge,dp
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,...,1JHC,84.8076,1.091953,0,c_h,2,0.919760,1,-0.401768,-3.655913e-01
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,...,2JHH,-11.2570,1.783120,3,h_h,11,0.648310,1,0.267843,8.915599e-07
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,...,2JHH,-11.2548,1.783147,3,h_h,11,0.648205,1,0.267844,1.783147e-06
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,...,2JHH,-11.2543,1.783157,3,h_h,11,0.648170,1,0.267844,1.783157e-06
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,...,1JHC,84.8074,1.091952,0,c_h,2,0.919760,1,-0.401767,-3.655914e-01


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,type,scalar_coupling_constant,distance,type_enc,pairs,pairs_enc,bond,bond_int,total_charge,dp
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,...,1JHC,84.8076,1.091953,0,c_h,2,0.919760,1,-0.401768,-3.655913e-01
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,...,2JHH,-11.2570,1.783120,3,h_h,11,0.648310,1,0.267843,8.915599e-07
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,...,2JHH,-11.2548,1.783147,3,h_h,11,0.648205,1,0.267844,1.783147e-06
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,...,2JHH,-11.2543,1.783157,3,h_h,11,0.648170,1,0.267844,1.783157e-06
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,...,1JHC,84.8074,1.091952,0,c_h,2,0.919760,1,-0.401767,-3.655914e-01


## Bond angle, dihedral angle

# Feature Engineering (test)

## Mulliken Charge (Imputation)

In [41]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,2JHC
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,1JHC
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,NaN,3JHH
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,1JHC
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,2JHC


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,2JHC
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,1JHC
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,NaN,3JHH
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,1JHC
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,2JHC


In [42]:
avg_charge=pd.DataFrame(train.groupby('atom_1')['mulliken_charge_1'].mean()).reset_index()
avg_charge.head()

,atom_1,mulliken_charge_1
0,C,-0.064872
1,H,0.118187
2,N,-0.269914


,atom_1,mulliken_charge_1
0,C,-0.064872
1,H,0.118187
2,N,-0.269914


In [43]:
test.drop('mulliken_charge_1',axis=1,inplace=True)
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,2JHC
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,1JHC
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,3JHH
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,1JHC
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,2JHC


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,2JHC
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,1JHC
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,3JHH
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,1JHC
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,2JHC


In [44]:
test=pd.merge(test,avg_charge,on='atom_1',how='left')
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,2JHC,-0.064872
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,1JHC,-0.064872
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,3JHH,0.118187
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,1JHC,-0.064872
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,2JHC,-0.064872


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,2JHC,-0.064872
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,1JHC,-0.064872
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,3JHH,0.118187
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,1JHC,-0.064872
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,2JHC,-0.064872


In [45]:
#only H is present
test['mulliken_charge_0']=train['mulliken_charge_0'].mean()
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,2JHC,-0.064872
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,1JHC,-0.064872
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,0.12211,3JHH,0.118187
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,1JHC,-0.064872
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,2JHC,-0.064872


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,2JHC,-0.064872
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,1JHC,-0.064872
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,0.12211,3JHH,0.118187
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,1JHC,-0.064872
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,2JHC,-0.064872


## Euclidian Distance

In [15]:
test['distance']=get_distance(test)
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,distance
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,2JHC,2.261178
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,1JHC,1.062099
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,NaN,3JHH,3.323277
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,1JHC,1.062099
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,2JHC,2.261178


## one-hot encode atom names
(Not neccessary)

In [47]:
#test=test.join(pd.get_dummies(test['atom_0'],prefix='atom_0'))
#test.head()

In [48]:
#test=test.join(pd.get_dummies(test['atom_1'],prefix='atom_1'))
#test.head()

## Label Encode type

In [22]:
test['type_enc']=le_type.transform(test['type'])
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,distance,type_enc
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,2JHC,2.261178,2
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,1JHC,1.062099,0
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,NaN,3JHH,3.323277,6
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,1JHC,1.062099,0
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,2JHC,2.261178,2


## Pairs

In [50]:
test['pairs']=(test['atom_0']+'_'+test['atom_1']).apply(lambda x:x.split('_')).apply(sorted).apply(lambda x:'_'.join(x)).apply(str.lower)

In [51]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,pairs
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,0.12211,3JHH,0.118187,3.323277,6,h_h
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,pairs
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,0.12211,3JHH,0.118187,3.323277,6,h_h
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h


## Pairs Label Encode

In [52]:
pairs_dict

{'c_c': 0,
 'c_f': 1,
 'c_h': 2,
 'c_n': 3,
 'c_o': 4,
 'c_si': 5,
 'f_f': 6,
 'f_h': 7,
 'f_n': 8,
 'f_o': 9,
 'f_si': 10,
 'h_h': 11,
 'h_n': 12,
 'h_o': 13,
 'h_si': 14,
 'n_n': 15,
 'n_o': 16,
 'o_o': 17}

{'c_c': 0,
 'c_f': 1,
 'c_h': 2,
 'c_n': 3,
 'c_o': 4,
 'c_si': 5,
 'f_f': 6,
 'f_h': 7,
 'f_n': 8,
 'f_o': 9,
 'f_si': 10,
 'h_h': 11,
 'h_n': 12,
 'h_o': 13,
 'h_si': 14,
 'n_n': 15,
 'n_o': 16,
 'o_o': 17}

In [53]:
test['pairs'].unique()

array(['c_h', 'h_h', 'h_n'], dtype=object)

array(['c_h', 'h_h', 'h_n'], dtype=object)

In [54]:
test['pairs_enc']=test['pairs'].map(pairs_dict)

In [55]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,pairs,pairs_enc
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h,2
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h,2
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,0.12211,3JHH,0.118187,3.323277,6,h_h,11
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h,2
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h,2


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,pairs,pairs_enc
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h,2
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h,2
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,0.12211,3JHH,0.118187,3.323277,6,h_h,11
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h,2
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h,2


## Connectivity

In [56]:
x=test[['pairs_enc','distance']]

In [57]:
x.head()

,pairs_enc,distance
0,2,2.261178
1,2,1.062099
2,11,3.323277
3,2,1.062099
4,2,2.261178


,pairs_enc,distance
0,2,2.261178
1,2,1.062099
2,11,3.323277
3,2,1.062099
4,2,2.261178


In [58]:
x.shape

(2505542, 2)

(2505542, 2)

In [59]:
test['bond']=model_nn.predict(x,verbose=1,batch_size=10000)

2505542/2505542 [==============================] - 49s 20us/sample


In [60]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,pairs,pairs_enc,bond
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,0.12211,3JHH,0.118187,3.323277,6,h_h,11,0.173375
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,pairs,pairs_enc,bond
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,0.12211,3JHH,0.118187,3.323277,6,h_h,11,0.173375
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226


In [61]:
test.shape

(2505542, 20)

(2505542, 20)

In [62]:
test['bond'].isna().value_counts()

False    2505542
Name: bond, dtype: int64

False    2505542
Name: bond, dtype: int64

In [63]:
test['bond_int']=test['bond'].apply(lambda x:int(x>0.5))

In [64]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,z_1,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,pairs,pairs_enc,bond,bond_int
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,...,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,...,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,...,1.0,0.12211,3JHH,0.118187,3.323277,6,h_h,11,0.173375,0
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,...,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,...,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,z_1,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,pairs,pairs_enc,bond,bond_int
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,...,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,...,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,...,1.0,0.12211,3JHH,0.118187,3.323277,6,h_h,11,0.173375,0
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,...,1.0,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,...,1.0,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0


## Total Charge

In [65]:
test['total_charge']=test['mulliken_charge_0']+test['mulliken_charge_1']

In [66]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,pairs,pairs_enc,bond,bond_int,total_charge
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,...,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0,0.057238
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,...,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1,0.057238
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,...,0.12211,3JHH,0.118187,3.323277,6,h_h,11,0.173375,0,0.240297
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,...,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1,0.057238
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,...,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0,0.057238


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,mulliken_charge_0,type,mulliken_charge_1,distance,type_enc,pairs,pairs_enc,bond,bond_int,total_charge
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,...,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0,0.057238
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,...,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1,0.057238
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,...,0.12211,3JHH,0.118187,3.323277,6,h_h,11,0.173375,0,0.240297
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,...,0.12211,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1,0.057238
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,...,0.12211,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0,0.057238


## Dipole Moment

In [67]:
test['dp']=(-1*test['mulliken_charge_0']*test['distance']/2)+(test['mulliken_charge_1']*test['distance']/2)

In [68]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,type,mulliken_charge_1,distance,type_enc,pairs,pairs_enc,bond,bond_int,total_charge,dp
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,...,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0,0.057238,-0.211400
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,...,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1,0.057238,-0.099297
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,...,3JHH,0.118187,3.323277,6,h_h,11,0.173375,0,0.240297,-0.006518
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,...,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1,0.057238,-0.099297
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,...,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0,0.057238,-0.211400


,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,...,type,mulliken_charge_1,distance,type_enc,pairs,pairs_enc,bond,bond_int,total_charge,dp
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,...,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0,0.057238,-0.211400
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,...,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1,0.057238,-0.099297
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,...,3JHH,0.118187,3.323277,6,h_h,11,0.173375,0,0.240297,-0.006518
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,...,1JHC,-0.064872,1.062099,0,c_h,2,0.917924,1,0.057238,-0.099297
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,...,2JHC,-0.064872,2.261178,2,c_h,2,0.413226,0,0.057238,-0.211400


# Drop & Rearrange Columns

In [23]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant,distance,type_enc
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076,1.091953,0
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570,1.783120,3
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548,1.783147,3
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543,1.783157,3
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074,1.091952,0


In [24]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,distance,type_enc
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,2JHC,2.261178,2
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,1JHC,1.062099,0
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,NaN,3JHH,3.323277,6
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,1JHC,1.062099,0
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,2JHC,2.261178,2


In [25]:
print('train: {}'.format(train.shape))
print('test: {}'.format(test.shape))

train: (4658147, 18)
test: (2505542, 17)


In [26]:
train.columns

Index(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'atom_0', 'x_0',
       'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1', 'mulliken_charge_0',
       'mulliken_charge_1', 'type', 'scalar_coupling_constant', 'distance',
       'type_enc'],
      dtype='object')

In [ ]:
drop_cols=['id', 'molecule_name',  'atom_0',  'atom_1', 'mulliken_charge_0',
       'mulliken_charge_1','type']

In [30]:
train.drop(drop_cols,axis=1,inplace=True)
train.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,scalar_coupling_constant,distance,type_enc
0,1,0,0.002150,-0.006031,0.001976,-0.012698,1.085804,0.008001,84.8076,1.091953,0
1,1,2,0.002150,-0.006031,0.001976,1.011731,1.463751,0.000277,-11.2570,1.783120,3
2,1,3,0.002150,-0.006031,0.001976,-0.540815,1.447527,-0.876644,-11.2548,1.783147,3
3,1,4,0.002150,-0.006031,0.001976,-0.523814,1.437933,0.906397,-11.2543,1.783157,3
4,2,0,1.011731,1.463751,0.000277,-0.012698,1.085804,0.008001,84.8074,1.091952,0


In [34]:
test.columns

Index(['atom_index_0', 'atom_index_1', 'x_0', 'y_0', 'z_0', 'x_1', 'y_1',
       'z_1', 'distance', 'type_enc'],
      dtype='object')

In [ ]:
drop_cols=['id', 'molecule_name',  'atom_0',  'atom_1', 'mulliken_charge_0',
       'mulliken_charge_1','type']

In [36]:
test.drop(drop_cols,axis=1,inplace=True)
test.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,distance,type_enc
0,2,0,-1.661639,0.0,1.0,0.599539,0.0,1.0,2.261178,2
1,2,1,-1.661639,0.0,1.0,-0.599539,0.0,1.0,1.062099,0
2,2,3,-1.661639,0.0,1.0,1.661639,0.0,1.0,3.323277,6
3,3,0,1.661639,0.0,1.0,0.599539,0.0,1.0,1.062099,0
4,3,1,1.661639,0.0,1.0,-0.599539,0.0,1.0,2.261178,2


In [76]:
train.isna().describe()

,scalar_coupling_constant,distance,type_enc,pairs_enc,bond_int,total_charge,dp
count,4658147,4658147,4658147,4658147,4658147,4658147,4658147
unique,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False
freq,4658147,4658147,4658147,4658147,4658147,4658147,4658147


,scalar_coupling_constant,distance,type_enc,pairs_enc,bond_int,total_charge,dp
count,4658147,4658147,4658147,4658147,4658147,4658147,4658147
unique,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False
freq,4658147,4658147,4658147,4658147,4658147,4658147,4658147


In [37]:
test.isna().describe()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,distance,type_enc
count,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542
unique,1,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False,False
freq,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542,2505542


In [38]:
train.columns

Index(['atom_index_0', 'atom_index_1', 'x_0', 'y_0', 'z_0', 'x_1', 'y_1',
       'z_1', 'scalar_coupling_constant', 'distance', 'type_enc'],
      dtype='object')

In [39]:
train=train[['atom_index_0', 'atom_index_1', 'x_0', 'y_0', 'z_0', 'x_1', 'y_1','z_1', 'distance', 'type_enc', 'scalar_coupling_constant']]
train.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,distance,type_enc,scalar_coupling_constant
0,1,0,0.002150,-0.006031,0.001976,-0.012698,1.085804,0.008001,1.091953,0,84.8076
1,1,2,0.002150,-0.006031,0.001976,1.011731,1.463751,0.000277,1.783120,3,-11.2570
2,1,3,0.002150,-0.006031,0.001976,-0.540815,1.447527,-0.876644,1.783147,3,-11.2548
3,1,4,0.002150,-0.006031,0.001976,-0.523814,1.437933,0.906397,1.783157,3,-11.2543
4,2,0,1.011731,1.463751,0.000277,-0.012698,1.085804,0.008001,1.091952,0,84.8074


In [41]:
test=test[['atom_index_0', 'atom_index_1', 'x_0', 'y_0', 'z_0', 'x_1', 'y_1','z_1', 'distance', 'type_enc']]
test.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,distance,type_enc
0,2,0,-1.661639,0.0,1.0,0.599539,0.0,1.0,2.261178,2
1,2,1,-1.661639,0.0,1.0,-0.599539,0.0,1.0,1.062099,0
2,2,3,-1.661639,0.0,1.0,1.661639,0.0,1.0,3.323277,6
3,3,0,1.661639,0.0,1.0,0.599539,0.0,1.0,1.062099,0
4,3,1,1.661639,0.0,1.0,-0.599539,0.0,1.0,2.261178,2


# Serialize Data

In [42]:
print(train.shape)
print(test.shape)

(4658147, 11)
(2505542, 10)


In [43]:
mid_point=int(train.shape[0]/2)
train[:mid_point].to_csv('train_feat_eng_v2_1.csv',index=False)
train[mid_point:].to_csv('train_feat_eng_v2_2.csv',index=False)

In [44]:
test.to_csv('test_feat_eng_v2.csv',index=False)

In [45]:
!ls

connectivity_model_nn_03.h5  test_etl.csv	   train_feat_eng_v2_1.csv
data.covs.pickle	     test_feat_eng_v2.csv  train_feat_eng_v2_2.csv
pairs_dict.pkl		     train_etl.csv


In [54]:
df=pd.read_csv('train_feat_eng_v2_2.csv')
df.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,distance,type_enc,scalar_coupling_constant
0,10,2,0.201991,-0.973581,-2.072494,0.680673,-0.974093,-1.091626,1.091439,0,94.12640
1,10,3,0.201991,-0.973581,-2.072494,0.051412,-1.899314,-0.020235,2.256418,2,-1.38365
2,10,4,0.201991,-0.973581,-2.072494,0.599497,-0.963231,1.103777,3.201065,5,11.84720
3,10,7,0.201991,-0.973581,-2.072494,-1.189860,-0.940525,0.066629,2.552291,5,4.28989
4,10,11,0.201991,-0.973581,-2.072494,1.770906,-0.986857,-1.167952,1.811041,3,-1.02528


In [55]:
df.shape

(2329074, 11)

In [56]:
df=pd.read_csv('test_feat_eng_v2.csv')
df.head()

,atom_index_0,atom_index_1,x_0,y_0,z_0,x_1,y_1,z_1,distance,type_enc
0,2,0,-1.661639,0.0,1.0,0.599539,0.0,1.0,2.261178,2
1,2,1,-1.661639,0.0,1.0,-0.599539,0.0,1.0,1.062099,0
2,2,3,-1.661639,0.0,1.0,1.661639,0.0,1.0,3.323277,6
3,3,0,1.661639,0.0,1.0,0.599539,0.0,1.0,1.062099,0
4,3,1,1.661639,0.0,1.0,-0.599539,0.0,1.0,2.261178,2


In [57]:
df.shape

(2505542, 10)

## Upload files to IBM Object Store

In [51]:
# The code was removed by Watson Studio for sharing.

In [52]:
from ibm_botocore.client import Config
import ibm_boto3
def upload_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.upload_file(Filename=local_file_name, Bucket=credentials['BUCKET'],Key=key)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Uploaded')

In [59]:
upload_file_cos(credentials_2,'train_feat_eng_v2_1.csv','train_feat_eng_v2_1.csv')
upload_file_cos(credentials_2,'train_feat_eng_v2_2.csv','train_feat_eng_v2_2.csv')

File Uploaded
File Uploaded


In [60]:
upload_file_cos(credentials_2,'test_feat_eng_v2.csv','test_feat_eng_v2.csv')

File Uploaded
